In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "WF"
BARK_VAR = "CA"
FIA_SPCD = 15

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(-0.1593, 0.8911, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,7478.00000,7.478000e+03,7478.0,7478.000000,7478.000000,7478.000000,7478.000000,7478.000000,7478.000000,7478.000000,...,365.000000,364.000000,7478.000000,7478.000000,7478.000000,7478.000000,7478.0,7478.000000,7478.000000,7478.000000
mean,560.03410,2.279706e+11,15.0,133.517954,0.261423,2.862578,56.689489,15.698689,0.577247,0.547827,...,0.289041,0.596291,2004.824819,7.591736,2014.824819,7.645360,10.0,3.889676,16.047205,551.502541
std,53.86656,1.749225e+11,0.0,77.726379,0.177286,1.924303,11.182074,10.368679,0.197450,0.195958,...,0.208581,0.403301,2.571604,1.518050,2.571604,1.575419,0.0,2.124013,9.195151,333.036875
min,417.00000,6.010156e+10,15.0,10.000000,0.000000,0.000000,11.000000,5.000000,0.020000,0.000000,...,0.050000,0.050000,2001.000000,1.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,511.00000,6.030216e+10,15.0,74.818920,0.120000,1.047198,49.000000,7.300000,0.400000,0.400000,...,0.150000,0.300000,2003.000000,7.000000,2013.000000,7.000000,10.0,2.000000,7.000000,246.000000
50%,517.00000,6.051096e+10,15.0,116.660000,0.250000,2.967060,57.000000,11.600000,0.600000,0.550000,...,0.250000,0.500000,2005.000000,7.000000,2015.000000,8.000000,10.0,3.000000,14.000000,566.000000
75%,610.00000,4.103036e+11,15.0,169.773500,0.350000,4.607669,65.000000,24.300000,0.717500,0.700000,...,0.400000,0.800000,2007.000000,9.000000,2017.000000,9.000000,10.0,7.000000,25.000000,840.000000
max,711.00000,5.308078e+11,15.0,523.332582,1.280000,6.283185,91.000000,71.100000,0.990000,0.990000,...,1.300000,2.400000,2009.000000,12.000000,2019.000000,11.000000,10.0,7.000000,32.000000,1126.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [10:47<00:00,  3.09it/s, 127 steps of size 3.40e-02. a


Done.
Samples saved at ../../models/maicf/WF//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WF//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [19:12<00:00,  1.74it/s, 127 steps of size 2.90e-02. a


Done.
Samples saved at ../../models/maicf/WF//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WF//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/WF//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WF//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [10:51<00:00,  3.07it/s, 127 steps of size 3.47e-02. a


Done.
Samples saved at ../../models/maicf/WF//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WF//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [19:06<00:00,  1.74it/s, 127 steps of size 3.30e-02. a


Done.
Samples saved at ../../models/maicf/WF//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WF//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [08:12<00:00,  4.06it/s, 127 steps of size 3.25e-02. a


Done.
Samples saved at ../../models/maicf/WF//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/WF//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [08:17<00:00,  4.02it/s, 127 steps of size 3.09e-02. a


Done.
Samples saved at ../../models/maicf/WF//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WF//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [17:57<00:00,  1.86it/s, 255 steps of size 2.75e-02. a


Done.
Samples saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [13:19<00:00,  2.50it/s, 127 steps of size 3.27e-02. a


Done.
Samples saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/WF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
